# Loading the data into the DB, using QueryGenerators (3/3).

Following the concepts on the different [means](/HowToGuides/start/Ingestion) of ingesting the data, we will build an example using the DataModel method in this notebook.

We will use the [Cookbook dataset](https://docs.google.com/spreadsheets/d/1G1HPG3Dxx5W39OD6b74wMHvWupD7N-DLUbV7tD5owx8/edit?usp=sharing) to have the data persisted onto ApertureDB instance.

We merge the first 3 sheets on the source into a json file such that there will be a list of Dishes objects, and each Dish may have multiple ingredients, along with some oher miscellaneous properties.



# Ingest using a Query Generator

When the source data is in a format that does not conform to any of the CSV pasrsers in the SDK, we could use the approach of defining a custom Query generator.

This does require a level of familiarity with the Query language.

Let's implement a class to deal with the cookbook example.

The Query generator is used to define a ```getitem``` to return a query to issue to ApertureDB that persists the record being iterated at the source.

In [ ]:
from typing import Tuple
from aperturedb.QueryGenerator import QueryGenerator
from aperturedb.types import *
from aperturedb.Sources import Sources
import json

class CookBookQueryGenerator(QueryGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__()
        assert "dishes" in kwargs, "Path to Dishes must be provided"
        with open(kwargs["dishes"]) as ins:
            self.dishes = self.dishes = json.load(ins)
            print(f"Loaded {len(self.dishes)} dishes")

    def __len__(self) -> int:
        return len(self.dishes)

    def getitem(self, idx: int) -> Tuple[Commands, Blobs]:
        record = self.dishes[idx]
        q = [
            {
                "AddImage":{
                    "_ref": 1,
                    "properties": {
                        "contributor": record["contributor"],
                        "name": record["name"],
                        "location": record["location"],
                        "cuisine": record["cuisine"],
                        "caption": record["caption"],
                        "recipe_url": record["recipe_url"],
                        "dish_id": record["dish_id"]
                    }
                }
            }
        ]
        for i, ingredient in enumerate(record["ingredients"]):
            q.append({
                "AddEntity": {
                    "_ref": 2 + i,
                    "class": "Ingredient",
                    "connect": {
                        "ref": 1
                    },
                    "properties": {
                        "Name": ingredient["Name"],
                        "other_names": ingredient["other_names"],
                        "macronutrient": ingredient["macronutrient"],
                        "micronutrient": ingredient["micronutrient"],
                        "subgroup": ingredient["subgroup"],
                        "category": ingredient["category"]
                    }
                }
            })

        blob = Sources(n_download_retries=3).load_from_http_url(record["url"], validator=lambda x: True)
        return q, [blob[1]]


from aperturedb.CommonLibrary import create_connector, execute_query

client = create_connector()
generator = CookBookQueryGenerator(dishes="dishes.json")
for query, blobs in generator:
    result, response, output_blobs = execute_query(client, query, blobs)

    if result != 0:
        print(response, query)
        break